In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install -q scipy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
model_id = 'EleutherAI/gpt-neo-125m'

In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neo-1.3B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

In [17]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [18]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [19]:
from datasets import load_dataset

data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [20]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.410600
2,2.533600
3,2.858100
4,3.415700
5,2.694400
6,2.463400
7,2.569800
8,2.674500
9,2.571200
10,2.833100


TrainOutput(global_step=10, training_loss=2.7024439334869386, metrics={'train_runtime': 6.0993, 'train_samples_per_second': 6.558, 'train_steps_per_second': 1.64, 'total_flos': 6345325240320.0, 'train_loss': 2.7024439334869386, 'epoch': 0.02})

In [21]:
trainer.save_model('adapter-model')


In [23]:
from peft import PeftModel    

base_model = AutoModelForCausalLM.from_pretrained(
        'EleutherAI/gpt-neo-1.3B',
        return_dict=True,
        torch_dtype=torch.float16 
)

model = PeftModel.from_pretrained(base_model, 'adapter-model')
print(f"Running merge_and_unload")
model.to('cuda')
model = model.merge_and_unload() 
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-1.3B')

model.save_pretrained("merged")
tokenizer.save_pretrained("merged")


Running merge_and_unload


('merged/tokenizer_config.json',
 'merged/special_tokens_map.json',
 'merged/vocab.json',
 'merged/merges.txt',
 'merged/added_tokens.json',
 'merged/tokenizer.json')

In [28]:
model_id = "merged"

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)




In [30]:
text = "Gravity is"
inputs = tokenizer(text, return_tensors="pt").to("cuda:0")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Gravity is a force that pulls you towards the center of the Earth.

Gravity is a force that pulls you towards the center of the Earth.

Gravity is a force that pulls you towards the center of the Earth.

Gravity


In [33]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
)

model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

In [35]:


outputs = model_nf4.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Gravity is a force that pulls everything in the universe together. It is the force that holds the Earth in place, and it is the force that holds the Sun in place. It is the force that holds the Moon in place, and it is the force that


In [43]:
double_quant_nf4_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant=True
)

model_double = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=double_quant_nf4_config)
outputs = model_double.generate(**inputs, max_new_tokens=200)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[   38, 16995,   318,   257,  2700,   326, 16194,  2279,   287,   262,
          6881,  1978,    13,   632,   318,   262,  2700,   326,  6622,   262,
          3668,   287,  1295,    11,   290,   340,   318,   262,  2700,   326,
          6622,   262,  3825,   287,  1295,    13,   632,   318,   262,  2700,
           326,  6622,   262,  6869,   287,  1295,    11,   290,   340,   318,
           262,  2700,   326,  6622,   262,  3668,   287,  1295,    13,   632,
           318,   262,  2700,   326,  6622,   262,  3825,   287,  1295,    11,
           290,   340,   318,   262,  2700,   326,  6622,   262,  3668,   287,
          1295,    13,   632,   318,   262,  2700,   326,  6622,   262,  6869,
           287,  1295,    11,   290,   340,   318,   262,  2700,   326,  6622,
           262,  3668,   287,  1295,    13,   632,   318,   262,  2700,   326,
          6622,   262,  3825,   287,  1295,    11,   290,   340,   318,   262,
          2700,   326,  6622,   262,  3668,   287,  